<a href="https://colab.research.google.com/github/ATOMScience-org/AMPL/blob/master/atomsci/ddm/examples/tutorials/11_CHEMBL26_SCN5A_IC50_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Building a Graph Convolutional Network Model for Drug Response Prediction</h1>

The ATOM Modeling PipeLine (AMPL; https://github.com/ATOMScience-org/AMPL) is an open-source, modular, extensible software pipeline for building and sharing models to advance in silico drug discovery.


## Time to run: 6 minutes

In [ ]:
!date

Thu Sep  9 22:04:01 UTC 2021


## Change your runtime to GPU 

Go to **Runtime** --> Change **runtime type** to "GPU"


## Goal: Use AMPL for predicting binding affinities -IC50 values- of ligands that could bind to human **Sodium channel protein type 5 subunit alpha** protein using Graph Convolutional Network Model. ChEMBL database is the source of the binding affinities (pIC50).

In this notebook, we describe the following steps using AMPL:

1.   Read a ML ready dataset
2.   Fit a Graph Convolutional model
3.   Predict pIC50 values of withheld compounds

## Set up
We first import the AMPL modules for use in this notebook.

The relevant AMPL modules for this example are listed below:

|module|Description|
|-|-|
|`atomsci.ddm.pipeline.model_pipeline`|The model pipeline module is used to fit models and load models for prediction.|
|`atomsci.ddm.pipeline.parameter_parser`|The parameter parser reads through pipeline options for the model pipeline.|
|`atomsci.ddm.utils.curate_data`|The curate data module is used for data loading and pre-processing.|
|`atomsci.ddm.utils.struct_utils`|The structure utilities module is used to process loaded structures.|


import atomsci.ddm.pipeline.compare_models as cmp
import atomsci.ddm.pipeline.model_pipeline as mp
import atomsci.ddm.pipeline.parameter_parser as parse

## Install AMPL 

In [ ]:
! pip install rdkit-pypi
! pip install deepchem 

import deepchem
# print(deepchem.__version__)
! pip install umap
! pip install -U --ignore-installed numba
! pip install umap-learn
! pip install molvs
! pip install bravado

In [ ]:
import deepchem as dc

# get the Install AMPL_GPU_test.sh
!wget 'https://raw.githubusercontent.com/ATOMScience-org/AMPL/master/atomsci/ddm/examples/tutorials/config/install_AMPL_GPU_test.sh'

# run the script to install AMPL
! chmod u+x install_AMPL_GPU_test.sh
! ./install_AMPL_GPU_test.sh

In [ ]:
dc.__version__

'2.6.0.dev'

In [ ]:
# We temporarily disable warnings for demonstration.
# FutureWarnings and DeprecationWarnings are present from some of the AMPL 
# dependency modules.
import warnings
warnings.filterwarnings('ignore')
import json
import requests
import sys

In [ ]:
import atomsci.ddm.pipeline.compare_models as cmp
import atomsci.ddm.pipeline.model_pipeline as mp
import atomsci.ddm.pipeline.parameter_parser as parse

2021-09-09 22:05:08,675 Model tracker client not supported in your environment; can look at models in filesystem only.


In [ ]:
import os
os.mkdir('chembl_activity_models')

**Let us display the dataset**

In [ ]:
import pandas as pd
import requests
import io
url = 'https://raw.githubusercontent.com/ATOMScience-org/AMPL/master/atomsci/ddm/examples/tutorials/datasets/ChEMBL26_SCN5A_IC50_human_ml_ready.csv'
download = requests.get(url).content
df = pd.read_csv(url, index_col=0)
# Reading the downloaded content and turning it into a pandas dataframe
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df.iloc[0:5, 0:5]
df.to_csv('ChEMBL26_SCN5A_IC50_human_ml_ready.csv', index=False)

In [ ]:
df

,base_rdkit_smiles,compound_id,pIC50,relation,active
0,O=S(=O)(Nc1nccs1)c1ccc2c(c1)OCCN2c1ccc(Cl)cc1O...,CHEMBL3948159,5.806875,NaN,1
1,Fc1cccc(Cn2ccc3c(OC4CCN(Cc5ccccn5)CC4)ncnc32)c1F,CHEMBL2012299,7.698970,NaN,1
2,COc1cc(-c2cccc(F)c2)c(Cl)cc1-c1ncc(O)c2cc(S(=O...,CHEMBL4089326,5.000000,<,0
3,O=C(Nc1cccc(C(F)(F)F)c1-c1ccccn1)c1cccc(-c2ncc...,CHEMBL4084926,5.721246,NaN,1
4,CC(C)COc1ncc(-c2nn(C)c3cc(C(=O)NS(C)(=O)=O)ccc...,CHEMBL4280298,5.186552,NaN,1
...,...,...,...,...,...
1724,O=C(NCc1ccc(C(F)(F)F)cc1)C1c2ccccc2C(=O)N1CCc1...,CHEMBL2164387,5.795880,NaN,1
1725,COc1ccc(Cl)c(-c2ccc(NC(=O)c3ccnn3C)nc2N)c1,CHEMBL3589904,4.431798,NaN,1
1726,N#Cc1cc(F)ccc1N1CCOc2cc(S(=O)(=O)Nc3nccs3)ccc21,CHEMBL3955931,5.000000,<,0
1727,Clc1cn(Cc2ccccc2)c2ncnc(OC3CCN(Cc4cscn4)CC3)c12,CHEMBL2012181,7.096910,NaN,1


base_splitter
Description:	Type of splitter to use for train/validation split if temporal split used for test set. May be random, scaffold, or ave_min. The allowable choices are set in splitter.py

Check here for details, https://github.com/ATOMScience-org/AMPL/blob/master/atomsci/ddm/docs/PARAMETERS.md

In [ ]:
split_config = {
"script_dir": "/content/AMPL/atomsci/ddm",
"dataset_key" : "/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv",
"datastore": "False",
"split_only": "True",
"splitter": "scaffold",
"split_valid_frac": "0.15",
"split_test_frac": "0.15",
"previously_split": "False",
"prediction_type": "regression",
"response_cols" : "pIC50",
"id_col": "compound_id",
"smiles_col" : "base_rdkit_smiles",
"result_dir": "/content/chembl_activity_models",
"system": "LC",
"transformers": "True",
"model_type": "NN",
"featurizer": "graphconv",
"descriptor_type": "graphconv",
"learning_rate": "0.0007",
"layer_sizes": "64,64,32",
"dropouts" : "0.0,0.0,0.0",
"save_results": "False",
"max_epochs": "100",
"verbose": "True"
}

In [ ]:
split_params = parse.wrapper(split_config)

In [ ]:
split_model = mp.ModelPipeline(split_params)

In [ ]:
split_uuid = split_model.split_dataset()
split_uuid

2021-09-09 22:05:42,367 Attempting to load featurized dataset
2021-09-09 22:05:42,389 Exception when trying to load featurized data:
DynamicFeaturization doesn't support get_featurized_dset_name()
2021-09-09 22:05:42,394 Featurized dataset not previously saved for dataset ChEMBL26_SCN5A_IC50_human_ml_ready, creating new
2021-09-09 22:05:42,412 Featurizing sample 0
2021-09-09 22:05:50,534 Featurizing sample 1000


number of features: 75


2021-09-09 22:05:57,308 Splitting data by scaffold
2021-09-09 22:06:11,467 Dataset split table saved to /content/ChEMBL26_SCN5A_IC50_human_ml_ready_train_valid_test_scaffold_c1043e32-61c9-46a4-bd86-3f004b1e8cfd.csv


'c1043e32-61c9-46a4-bd86-3f004b1e8cfd'

In [ ]:
!pip install --upgrade gspread

  Attempting uninstall: gspread
    Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


In [ ]:
!date

Tue Aug 10 23:46:59 UTC 2021


## Train the mode (~ 10 minutes)

In [ ]:
train_config = {
"script_dir": "/content/AMPL/atomsci/ddm",
"dataset_key" : "/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv",
"datastore": "False",
"uncertainty": "False",
"splitter": "scaffold",
"split_valid_frac": "0.15",
"split_test_frac": "0.15",
"previously_split": "True",
"split_uuid": "{}".format(split_uuid),
"prediction_type": "regression",
"response_cols" : "pIC50",
"id_col": "compound_id",
"smiles_col" : "base_rdkit_smiles",
"result_dir": "/content/chembl_activity_models",
"system": "LC",
"transformers": "True",
"model_type": "NN",
"featurizer": "graphconv",
"descriptor_type": "graphconv",
"learning_rate": "0.0007",
"layer_sizes": "64,64,32",
"dropouts" : "0.0,0.0,0.0",
"save_results": "False",
"max_epochs": "100",
"verbose": "True"
}

In [ ]:
train_params = parse.wrapper(train_config)

In [ ]:
train_model = mp.ModelPipeline(train_params)

## Train_model took ~ 18 minutes on a GPU  (~ 30 minutes on a CPU)

In [ ]:
mp.ampl_version

'1.2.0'

In [ ]:
dc.__version__

'2.6.0.dev'

In [ ]:
%cd github
!ls

/content/github
AMPL  AMPL.build  AMPL.dist  __init___py.patch	transformations_py.patch


In [ ]:
train_model.train_model()

2021-09-09 22:07:37,435 Attempting to load featurized dataset
2021-09-09 22:07:37,452 Exception when trying to load featurized data:
DynamicFeaturization doesn't support get_featurized_dset_name()
2021-09-09 22:07:37,453 Featurized dataset not previously saved for dataset ChEMBL26_SCN5A_IC50_human_ml_ready, creating new
2021-09-09 22:07:37,472 Featurizing sample 0
2021-09-09 22:07:45,336 Featurizing sample 1000


number of features: 75


2021-09-09 22:07:53,186 Previous dataset split restored
2021-09-09 22:07:53,219 Wrote transformers to /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/f06dbbd9-4416-4a5d-9d05-bbd6682f70e7/transformers.pkl
2021-09-09 22:07:53,220 Transforming response data
2021-09-09 22:07:54,148 Transforming response data
2021-09-09 22:07:54,311 Transforming response data
2021-09-09 22:08:18,151 Epoch 0: training r2_score = -0.263, validation r2_score = -0.629, test r2_score = -0.294
2021-09-09 22:08:19,189 Epoch 1: training r2_score = -0.205, validation r2_score = -0.707, test r2_score = -0.324
2021-09-09 22:08:20,044 Epoch 2: training r2_score = -0.108, validation r2_score = -0.776, test r2_score = -0.404
2021-09-09 22:08:20,877 Epoch 3: training r2_score = -0.174, validation r2_score = -0.581, test r2_score = -0.443
2021-09-09 22:08:21,876 Epoch 4: training r2_score = -0.019, validation r2_score = -0.321, test r2_score = -0.327
2021-09-09 22:08:22,8

In [ ]:
perf_df = cmp.get_filesystem_perf_results('/content/chembl_activity_models', pred_type='regression')
perf_df

Found data for 1 models under /content/chembl_activity_models


,model_uuid,model_path,ampl_version,model_type,dataset_key,featurizer,splitter,model_score_type,feature_transform_type,learning_rate,dropouts,layer_sizes,best_epoch,max_epochs,rf_estimators,rf_max_features,rf_max_depth,xgb_gamma,xgb_learning_rate,model_choice_score,train_r2_score,train_rms_score,train_mae_score,train_num_compounds,valid_r2_score,valid_rms_score,valid_mae_score,valid_num_compounds,test_r2_score,test_rms_score,test_mae_score,test_num_compounds
0,f06dbbd9-4416-4a5d-9d05-bbd6682f70e7,/content/chembl_activity_models/ChEMBL26_SCN5A...,1.2.0,NN,/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv,graphconv,scaffold,r2,normalization,0.0007,"0.00,0.00,0.00","64,64,32",63,100,NaN,NaN,NaN,NaN,NaN,0.101922,0.877191,0.224703,0.167855,1210,0.101922,0.556917,0.409377,259,0.161975,0.642576,0.453671,260


In [ ]:
!date

Thu Sep  9 22:12:39 UTC 2021
